In [1]:
import torch
import torch.nn as nn
from model import LocalPredictor

In [2]:
import os

In [3]:
import pickle as pk

In [4]:
import numpy as np

In [5]:
local_predictor = torch.load('./results_osaka/local_predictor_broader_5.pytorch').cuda(1)

In [6]:
date_list = [(2012, m, d) for m in range(12, 13) for d in range(1, 32)] + [(2013, m, d) for m in range(1, 3) for d in range(1, 32)]

In [7]:
T = 96
dT = 4
batch_size = 256

In [8]:
data = dict({})
for m in range(11, 12):
    for d in range(1, 29):
        filename = '/data/fan/UsersInOsakaProcessed/2012{:02d}{:02d}_interp.pk'.format(m, d)
        print(filename)
        with open(filename, 'rb') as f:
            data[(m, d)] = pk.load(f)

/data/fan/UsersInOsakaProcessed/20121101_interp.pk
/data/fan/UsersInOsakaProcessed/20121102_interp.pk
/data/fan/UsersInOsakaProcessed/20121103_interp.pk
/data/fan/UsersInOsakaProcessed/20121104_interp.pk
/data/fan/UsersInOsakaProcessed/20121105_interp.pk
/data/fan/UsersInOsakaProcessed/20121106_interp.pk
/data/fan/UsersInOsakaProcessed/20121107_interp.pk
/data/fan/UsersInOsakaProcessed/20121108_interp.pk
/data/fan/UsersInOsakaProcessed/20121109_interp.pk
/data/fan/UsersInOsakaProcessed/20121110_interp.pk
/data/fan/UsersInOsakaProcessed/20121111_interp.pk
/data/fan/UsersInOsakaProcessed/20121112_interp.pk
/data/fan/UsersInOsakaProcessed/20121113_interp.pk
/data/fan/UsersInOsakaProcessed/20121114_interp.pk
/data/fan/UsersInOsakaProcessed/20121115_interp.pk
/data/fan/UsersInOsakaProcessed/20121116_interp.pk
/data/fan/UsersInOsakaProcessed/20121117_interp.pk
/data/fan/UsersInOsakaProcessed/20121118_interp.pk
/data/fan/UsersInOsakaProcessed/20121119_interp.pk
/data/fan/UsersInOsakaProcessed

In [10]:
data_doc = dict({})

for date in data:
    for uid in data[date]:
        if uid not in data_doc:
            data_doc[uid] = [data[date][uid]]
        else:
            data_doc[uid].append(data[date][uid])

In [11]:
for uid in data_doc:
    data_doc[uid] = torch.LongTensor(data_doc[uid]).cuda(1)

In [13]:
mrr_avg_rank_dict = dict({})

for y, m, d in date_list:
    filename = '/data/fan/UsersInOsakaProcessed/{:04d}{:02d}{:02d}_interp.pk'.format(y, m, d)
    if not os.path.isfile(filename):
        continue
    print(filename)
    
    with open(filename, 'rb') as f:
        cnt_day = 0
        data = pk.load(f)
        for uid in data:
            data[uid] = torch.LongTensor([data[uid]]).cuda(1)
            
        mrr = 0.0
        avg_rank = 0.0
        acc5 = 0.0
        
        with torch.no_grad():
            for t in range(T - 2 * dT):
                
                for uid in data:
                    if np.random.ranf() < 0.05:
                        cnt_day += 1
                        data_x = data[uid][:, t: t + dT]
                        data_t = torch.zeros_like(data_x) + t
                        data_y = data[uid][:, t + 2 * dT - 1].cpu().numpy()
                        if uid in data_doc:
                            x_loc_doc = []
                            x_t_doc = []
                            for j in range(-2, 3):
                                if t + j >= 0 and t + 2 * dT + j <= T:
                                    tmp = data_doc[uid][:, t + j: t + 2 * dT + j]
                                    x_loc_doc.append(tmp)
                                    x_t_doc.append(torch.zeros_like(tmp) + t + j)
                            pred = local_predictor.predict(data_x, data_t, torch.cat(x_loc_doc, dim=0), torch.cat(x_t_doc, dim=0)).cpu().numpy()
                        else:
                            pred = local_predictor.predict(data_x, data_t, None, None).cpu().numpy()
                            
                        rank = np.where((np.argsort(-pred, axis=1, ).T - data_y).T == 0)[1] + 1
                        mrr += np.sum(1 / rank)
                        avg_rank += np.sum(rank)
                        acc5 += np.sum(rank <= 5)
                        
            print('Evaluation: mrr = {:.4f}, avg_rank = {:.4f}, acc@5 = {:.4f}'.format(mrr / cnt_day, avg_rank / cnt_day, acc5 / cnt_day))
            mrr_avg_rank_dict[(y, m, d)] = (mrr / cnt_day, avg_rank / cnt_day, acc5 / cnt_day)

/data/fan/UsersInOsakaProcessed/20121201_interp.pk
Evaluation: mrr = 0.8157, avg_rank = 18.6156, acc@5 = 0.8766
/data/fan/UsersInOsakaProcessed/20121202_interp.pk
Evaluation: mrr = 0.8261, avg_rank = 17.4751, acc@5 = 0.8822
/data/fan/UsersInOsakaProcessed/20121203_interp.pk
Evaluation: mrr = 0.8321, avg_rank = 15.2469, acc@5 = 0.8936
/data/fan/UsersInOsakaProcessed/20121204_interp.pk
Evaluation: mrr = 0.8293, avg_rank = 15.6767, acc@5 = 0.8909
/data/fan/UsersInOsakaProcessed/20121205_interp.pk
Evaluation: mrr = 0.8268, avg_rank = 15.6012, acc@5 = 0.8891
/data/fan/UsersInOsakaProcessed/20121206_interp.pk
Evaluation: mrr = 0.8272, avg_rank = 16.2155, acc@5 = 0.8888
/data/fan/UsersInOsakaProcessed/20121207_interp.pk
Evaluation: mrr = 0.8209, avg_rank = 16.6445, acc@5 = 0.8839
/data/fan/UsersInOsakaProcessed/20121208_interp.pk
Evaluation: mrr = 0.8146, avg_rank = 19.0128, acc@5 = 0.8747
/data/fan/UsersInOsakaProcessed/20121209_interp.pk
Evaluation: mrr = 0.8236, avg_rank = 18.4956, acc@5 =

Evaluation: mrr = 0.8187, avg_rank = 23.5930, acc@5 = 0.8761
/data/fan/UsersInOsakaProcessed/20130213_interp.pk
Evaluation: mrr = 0.8144, avg_rank = 24.1715, acc@5 = 0.8730
/data/fan/UsersInOsakaProcessed/20130214_interp.pk
Evaluation: mrr = 0.8139, avg_rank = 23.6904, acc@5 = 0.8728
/data/fan/UsersInOsakaProcessed/20130215_interp.pk
Evaluation: mrr = 0.8098, avg_rank = 24.1917, acc@5 = 0.8694
/data/fan/UsersInOsakaProcessed/20130216_interp.pk
Evaluation: mrr = 0.8098, avg_rank = 25.8269, acc@5 = 0.8656
/data/fan/UsersInOsakaProcessed/20130217_interp.pk
Evaluation: mrr = 0.8240, avg_rank = 23.8945, acc@5 = 0.8767
/data/fan/UsersInOsakaProcessed/20130218_interp.pk
Evaluation: mrr = 0.8182, avg_rank = 24.5723, acc@5 = 0.8753
/data/fan/UsersInOsakaProcessed/20130219_interp.pk
Evaluation: mrr = 0.8158, avg_rank = 25.1466, acc@5 = 0.8736
/data/fan/UsersInOsakaProcessed/20130220_interp.pk
Evaluation: mrr = 0.8120, avg_rank = 24.9308, acc@5 = 0.8722
/data/fan/UsersInOsakaProcessed/20130221_in

In [14]:
with open('./results_osaka/evaluation_local_broader_5_rank.pk', 'wb') as f:
    pk.dump(mrr_avg_rank_dict, f)